# Template - Strategy - Backtesting 

### Import Library

In [47]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
# from backtesting._plotting import set_bokeh_output
# set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [48]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

remote


In [49]:
data = dataset.copy()

In [50]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [51]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-13 14:15:00,1333.5,1335.4,1333.4,1335.0,5583
2025-02-13 14:20:00,1335.4,1336.5,1334.7,1334.9,5753
2025-02-13 14:25:00,1335.3,1336.8,1334.9,1336.8,4879


In [52]:
def get_1st_condition(r):
    signal = ''
    if r['Close'] > r['Close_s1'] > r['Close_s2']:
        # close price increasing
        signal = 'long'
    elif r['Close'] < r['Close_s1'] < r['Close_s2']:
        # close price decreasing
        signal = 'short'
    return signal

def get_2nd_condition(r):
    signal = ''
    if r['open_close'] > 0 and r['open_close_s1'] > 0 and r['open_close_s2'] > 0:
        # 3 green candlesticks
        signal = 'long'
    elif r['open_close'] < 0 and r['open_close_s1'] < 0 and r['open_close_s2'] < 0:
        # 3 red candlesticks
        signal = 'short'
    return signal

def get_3rd_condition(r):
    reward_on_risk = 3
    signal = ''
    if r['open_close'] > 0:
        risk = r['Close'] - r['min_4']
        posible_reward = r['max_12'] - r['Close']
        if posible_reward > reward_on_risk * risk:
            signal = 'long'
    elif r['open_close'] < 0:
        risk = r['max_4'] - r['Close']
        posible_reward = r['Close'] - r['min_12']
        if posible_reward > reward_on_risk * risk:
            signal = 'short'
    return signal

    
def cal_signal(r):
    signal = ''
    if r['condition_1'] == 'long' and r['condition_2'] == 'long' and r['condition_3'] == 'long':
        signal = 'long'
    elif r['condition_1'] == 'short' and r['condition_2'] == 'short' and r['condition_3'] == 'short':
        signal = 'short'
    return signal

def prepare_data(data):
    data['Close_s1'] = data['Close'].shift(1)
    data['Close_s2'] = data['Close'].shift(2)
    data['Close_s3'] = data['Close'].shift(3)
    data['open_close'] = data['Close'] - data['Open']
    data['open_close_s1'] = data['open_close'].shift(1)
    data['open_close_s2'] = data['open_close'].shift(2)
    data['max_4'] = data['High'].rolling(4).max()
    data['min_4'] = data['Low'].rolling(4).min()
    data['max_12'] = data['High'].rolling(12).max()
    data['min_12'] = data['Low'].rolling(12).min()
    data['condition_1'] = data.apply(lambda r: get_1st_condition(r), axis=1)
    data['condition_2'] = data.apply(lambda r: get_2nd_condition(r), axis=1)
    data['condition_3'] = data.apply(lambda r: get_3rd_condition(r), axis=1)
    data['signal'] = data.apply(lambda r: cal_signal(r), axis=1)
    return data

In [53]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [54]:
prepared_data[prepared_data.signal != '']

,Open,High,Low,Close,Volume,Close_s1,Close_s2,Close_s3,open_close,open_close_s1,open_close_s2,max_4,min_4,max_12,min_12,condition_1,condition_2,condition_3,signal
Date,,,,,,,,,,,,,,,,,,,
2020-11-03 09:20:00,908.5,908.8,908.1,908.1,847,908.5,908.6,908.8,-0.4,-0.1,-0.2,908.9,908.1,909.4,896.6,short,short,short,short
2020-11-03 09:25:00,908.2,908.6,907.7,907.8,1999,908.1,908.5,908.6,-0.4,-0.4,-0.1,908.9,907.7,909.4,896.6,short,short,short,short
2020-11-10 09:25:00,933.3,933.3,932.7,933.0,2165,933.2,934.0,934.0,-0.3,-0.8,-0.1,934.3,932.7,935.3,916.5,short,short,short,short
2020-11-16 09:30:00,941.5,941.6,940.9,941.0,2144,941.5,942.0,942.5,-0.5,-0.5,-0.5,942.6,940.9,942.6,932.5,short,short,short,short
2020-11-20 09:15:00,949.4,949.5,948.5,948.9,1923,949.3,950.8,951.1,-0.5,-1.4,-0.7,951.5,948.5,951.5,938.0,short,short,short,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-17 09:20:00,1309.1,1309.5,1307.7,1309.0,4876,1309.1,1311.4,1312.2,-0.1,-2.3,-0.8,1312.7,1307.7,1312.7,1295.1,short,short,short,short
2025-01-22 09:20:00,1327.0,1327.4,1325.8,1326.8,4878,1327.0,1327.1,1328.0,-0.2,-0.2,-0.8,1328.7,1325.3,1328.7,1318.2,short,short,short,short
2025-01-22 09:30:00,1325.9,1326.3,1325.1,1325.5,2619,1325.9,1326.8,1327.0,-0.4,-1.0,-0.2,1327.5,1325.1,1328.7,1318.2,short,short,short,short


In [55]:
prepared_data[(prepared_data.condition_1 != '') & (prepared_data.condition_2 != '')]

,Open,High,Low,Close,Volume,Close_s1,Close_s2,Close_s3,open_close,open_close_s1,open_close_s2,max_4,min_4,max_12,min_12,condition_1,condition_2,condition_3,signal
Date,,,,,,,,,,,,,,,,,,,
2020-11-02 14:05:00,901.5,901.7,897.0,897.9,3890,901.5,902.0,902.5,-3.6,-0.3,-0.5,903.0,897.0,903.0,896.3,short,short,,
2020-11-02 14:10:00,898.0,898.5,896.6,897.7,7216,897.9,901.5,902.0,-0.3,-3.6,-0.3,902.9,896.6,903.0,896.3,short,short,,
2020-11-02 14:25:00,904.9,907.0,904.5,907.0,5873,904.5,899.0,897.7,2.1,5.5,1.6,907.0,896.6,907.0,896.6,long,long,,
2020-11-03 09:20:00,908.5,908.8,908.1,908.1,847,908.5,908.6,908.8,-0.4,-0.1,-0.2,908.9,908.1,909.4,896.6,short,short,short,short
2020-11-03 09:25:00,908.2,908.6,907.7,907.8,1999,908.1,908.5,908.6,-0.4,-0.4,-0.1,908.9,907.7,909.4,896.6,short,short,short,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-13 10:35:00,1327.4,1328.8,1327.4,1328.7,3455,1327.3,1327.0,1326.4,1.3,0.1,0.6,1328.8,1326.2,1331.5,1326.2,long,long,,
2025-02-13 10:40:00,1328.7,1329.8,1328.6,1329.8,4716,1328.7,1327.3,1327.0,1.1,1.3,0.1,1329.8,1326.4,1331.5,1326.2,long,long,,
2025-02-13 11:10:00,1333.3,1333.6,1332.5,1333.5,3764,1333.3,1332.2,1330.5,0.2,1.1,1.8,1333.6,1330.4,1333.6,1326.2,long,long,,


In [56]:
class MainStrategy(Strategy):
    reward_on_risk = 3
    def init(self):
        self._broker._cash = 1500
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
            return

        if self.position:
            return 
        signal = self.data.signal[-1]
        close_price = self.data.Close[-1]
        open_price = self.data.Open[-1]
        min_4 = self.data.min_4[-1]
        max_4 = self.data.max_4[-1]
        if signal == 'long':
            buy_price = close_price
            sl = min_4
            tp = buy_price + self.reward_on_risk * (buy_price - min_4)
            self.buy(size=1, sl=sl, tp=tp)
        elif signal == 'short':
            sell_price = close_price
            sl = max_4
            tp = sell_price - self.reward_on_risk * (max_4 - sell_price)
            self.sell(size=1, sl=sl, tp=tp)

In [57]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [58]:
stats

Start                     2020-11-02 09:55:00
End                       2025-02-13 14:45:00
Duration                   1564 days 04:50:00
Exposure Time [%]                     6.15215
Equity Final [$]                    1709.0992
Equity Peak [$]                    1733.45437
Return [%]                          13.939947
Buy & Hold Return [%]                48.39676
Return (Ann.) [%]                    3.127144
Volatility (Ann.) [%]                2.859196
Sharpe Ratio                         1.093714
Sortino Ratio                        2.645612
Calmar Ratio                         0.954317
Max. Drawdown [%]                   -3.276841
Avg. Drawdown [%]                   -0.251735
Max. Drawdown Duration      314 days 23:35:00
Avg. Drawdown Duration       12 days 17:50:00
# Trades                                  222
Win Rate [%]                        37.387387
Best Trade [%]                       2.867263
Worst Trade [%]                     -1.250959
Avg. Trade [%]                    

In [59]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,-1,45,58,907.92754,908.9,-0.97246,-0.001071,2020-11-03 09:25:00,2020-11-03 10:30:00,None,0 days 01:05:00
1,-1,301,311,932.72010,929.1,3.62010,0.003881,2020-11-10 09:30:00,2020-11-10 10:20:00,None,0 days 00:50:00
2,-1,504,516,940.71770,936.2,4.51770,0.004802,2020-11-16 09:35:00,2020-11-16 10:35:00,None,0 days 01:00:00
3,-1,705,745,948.51536,951.5,-2.98464,-0.003147,2020-11-20 09:20:00,2020-11-20 14:05:00,None,0 days 04:45:00
4,-1,756,759,952.81407,950.4,2.41407,0.002534,2020-11-23 09:20:00,2020-11-23 09:35:00,None,0 days 00:15:00
...,...,...,...,...,...,...,...,...,...,...,...
217,-1,53559,53576,1295.61120,1298.2,-2.58880,-0.001998,2025-01-15 09:30:00,2025-01-15 10:55:00,None,0 days 01:25:00
218,-1,53660,53689,1308.60730,1312.7,-4.09270,-0.003128,2025-01-17 09:25:00,2025-01-17 13:15:00,None,0 days 03:50:00
219,-1,53813,53821,1326.50193,1321.1,5.40193,0.004072,2025-01-22 09:25:00,2025-01-22 10:05:00,None,0 days 00:40:00
220,-1,54020,54023,1321.50343,1324.2,-2.69657,-0.002041,2025-02-04 09:40:00,2025-02-04 09:55:00,None,0 days 00:15:00


In [60]:
# bt.plot()